In [45]:
from pgmpy.readwrite import BIFReader
import random

In [46]:
#Parameters
GROUP_ID = '04'
ALGORITHM = 'gibbs'
NETWORK_NAME = 'networks/child.bif'
REPORT = 'Disease'
EVIDENCE_LEVEL = 'None'
EVIDENCE = 'owerBodyO2="<5"; RUQO2=">=12"; CO2Report=">=7.5"; XrayReport=Asy/Patchy'

In [47]:
def gibbsSampling(Network, reportedVars, evidenceVars):
    possVals = Network.states
    randomVals = {}

    #Initializing random state values
    for key in possVals.keys():
        if key in evidenceVars.keys():
            randomVals[key] = evidenceVars[key]
        else:
            randomVals[key] = random.choice(possVals[key])
    print(randomVals)


In [48]:
reader = BIFReader(NETWORK_NAME)
model = reader.get_model()
reportVariables = REPORT.split(';')
evidenceVariables = {}
splitter = EVIDENCE.split(";")
for var in splitter:
    #Edge case for variables with '=' characters in their values
    if'"' in var:
        splitter2 = var.split('"')
        evidenceVariables[splitter2[0][:-1].strip()] = splitter2[1]
    else:
        splitter2 = var.split("=")
        evidenceVariables[splitter2[0].strip()] = splitter2[1]

if ALGORITHM == "gibbs":
    gibbsSampling(model,reportVariables,evidenceVariables)
elif ALGORITHM == "ve":
    print()
    #variable elim
else:
    print("Unrecognized algorithm:", ALGORITHM)



{'Disease': 'PAIVS', 'Sick': 'no', 'Age': '11-30_days', 'BirthAsphyxia': 'no', 'LungParench': 'Congested', 'CO2': 'High', 'CO2Report': '>=7.5', 'CardiacMixing': 'Transp.', 'LungFlow': 'Normal', 'ChestXray': 'Grd_Glass', 'DuctFlow': 'Rt_to_Lt', 'Grunting': 'yes', 'GruntingReport': 'yes', 'HypDistrib': 'Unequal', 'HypoxiaInO2': 'Severe', 'LVH': 'no', 'LVHreport': 'yes', 'LowerBodyO2': '<5', 'RUQO2': '>=12', 'XrayReport': 'Asy/Patchy'}
